In [1]:
import requests
import pandas as pd
from getpass import getpass
import os
import time
from pymongo import MongoClient
from bson.decimal128 import Decimal128
from collections import defaultdict
from dotenv import load_dotenv
load_dotenv()
import json
import geojson
from cartoframes.viz import Map, Layer, popup_element
import geopandas as gpd
import folium
from folium.plugins import HeatMap
from shapely.geometry import Polygon
from folium import Icon
from folium import plugins

## PROCESS AND SOLUTIONS ##

**IMPORT SOURCE DOCUMENTS**

In [2]:
import src.exploration as exp
import src.forsquare as fsq
import src.visualization as vis

**EXPLORATION FUNCTIONS**

In [3]:
client = MongoClient("localhost:27017")
db = client["ironhack"]
c = db.get_collection("companies")

    FINDING COMPANIES WITH 2 OF THE REQUIREMENTS OF EMPLOYEES:
    
        - design companies in the city
        - successful startups

In [6]:
design_companies = exp.find_design_companies(c)
successful_startups = exp.find_successful_startups(c)
city_counts = exp.oc_city_counts(successful_startups, "offices", "city")
city_counts_design= exp.oc_city_counts(design_companies, "offices", "city")

In [8]:
city_counts.head()

,Count
San Francisco,190
New York,127
London,78
Seattle,43
Palo Alto,41


In [9]:
city_counts_design

,Count
San Francisco,1
Collingwood,1
Berlin,1
London,1
Brooklyn,1
Ellensburg,1


FINDINGS

From what I found out, the top 3 cities to place the company are, San Francisco, New York, and London. 

In those cities there are the greatest number of successful startups and other Design companies to satisfy the Designers requirements.

But as the top 2 are in the States, instead of London I think it's better to find another city in the states to filter from. In that case the next option it's going to be Seattle. 

**GEOJSON AND 4SQR**

In [10]:
client = MongoClient("localhost:27017")
db = client["cities_project"]

In [11]:
token = os.getenv("token")

    GETTING LOCATION FOR EACH CITY

In [ ]:
san_francisco_loc = fsq.get_coordinates("501 Twin Peaks Blvd, San Francisco, CA 94114, Estados Unidos")
new_york_loc = fsq.get_coordinates("Central Park N, New York, New York")
seattle_loc = fsq.get_coordinates("1027 S Washington St, Seattle, WA 98104, EE. UU.")

In [19]:
san_francisco_loc = [37.75404,-122.44721]
new_york_loc = [40.79724,-73.94968]
seattle_loc = [47.60065,-122.31819]

    FINDING REQUIREMENTS FOR EACH CITY AND UPLOADING THEM TO MONGO

    - SAN FRANCISCO

In [ ]:
starbucks_SF = fsq.foursquare_places ("Starbucks",san_francisco_loc, "SFstarbucks")
basketball_stadiums_SF = fsq.foursquare_places ("Basketball stadium",san_francisco_loc, "SFbaskStadium")
schools_SF = fsq.foursquare_places ("schools", san_francisco_loc, "SFschools")
airport_SF = fsq.foursquare_places ("airport",san_francisco_loc,"SFairport")
train_stations_SF = fsq.foursquare_places ("train station",san_francisco_loc, "SFtrainStations")
pet_grooming_SF = fsq.foursquare_places ("pet grooming", san_francisco_loc,"SFpet_grooming")
dog_parks_SF = fsq.foursquare_places ("dog park", san_francisco_loc,"SFdog_parks")
bars_clubs_SF = fsq.foursquare_places ("night club",san_francisco_loc,"SFbars_clubs")
vegan_vegetarian_SF = fsq.foursquare_places ("vegan restaurant", san_francisco_loc,"SFvegan_vegetarian")

    - NEW YORK CITY

In [ ]:
starbucks_NYC = fsq.foursquare_places ("Starbucks",new_york_loc,"NYCstarbucks")
basketball_stadiums_NYC = fsq.foursquare_places ("Basketball stadium",new_york_loc,"NYCbaskStadium")
schools_NYC = fsq.foursquare_places ("schools",new_york_loc,"NYCschools")
airport_NYC = fsq.foursquare_places ("airport",new_york_loc,"NYCairport")
train_stations_NYC = fsq.foursquare_places ("train station",new_york_loc,"NYCtrain_stations")
pet_grooming_NYC = fsq.foursquare_places ("pet grooming",new_york_loc,"NYCpet_grooming")
dog_parks_NYC = fsq.foursquare_places ("dog park",new_york_loc,"NYCdog_parks")
bars_clubs_NYC = fsq.foursquare_places ("night club",new_york_loc,"NYCbars_clubs")
vegan_vegetarian_NYC = fsq.foursquare_places ("vegan restaurant",new_york_loc,"NYCvegan_vegetarian")

     - SEATTLE

In [ ]:
starbucks_ST = fsq.foursquare_places ("Starbucks",seattle_loc,"STstarbucks")
basketball_stadiums_ST = fsq.foursquare_places ("Basketball stadium",seattle_loc,"STbaskStadium")
schools_ST = fsq.foursquare_places ("schools",seattle_loc,"STschools")
airport_ST = fsq.foursquare_places ("airport",seattle_loc,"STairport")
train_stations_ST = fsq.foursquare_places ("train station",seattle_loc,"STtrain_stations")
pet_grooming_ST = fsq.foursquare_places ("pet grooming",seattle_loc, "STpet_grooming")
dog_parks_ST = fsq.foursquare_places ("dog park",seattle_loc,"STdog_parks")
bars_clubs_ST = fsq.foursquare_places ("night club",seattle_loc,"STbars_clubs")
vegan_vegetarian_ST = fsq.foursquare_places ("vegan restaurant",seattle_loc,"STvegan_vegetarian")

Once we've got all the collections uploaded to mongo we can start looking at the data and plotting.

**VISUALIZATION**

    IMPORTING MONGO COLLECTIONS OF REQUIREMENTS

    - SAN FRANCISCO

In [15]:
SFstarbucks = vis.mongodb_to_df("SFstarbucks")
SFbaskStadium = vis.mongodb_to_df("SFbaskStadium")
SFschools = vis.mongodb_to_df("SFschools")
SFairport = vis.mongodb_to_df("SFairport")
SFtrainStations = vis.mongodb_to_df("SFtrainStations")
SFpet_grooming = vis.mongodb_to_df("SFpet_grooming")
SFdog_parks = vis.mongodb_to_df("SFdog_parks")
SFbars_clubs = vis.mongodb_to_df("SFbars_clubs")
SFvegan_vegetarian = vis.mongodb_to_df("SFvegan_vegetarian")

    - NEW YORK CITY

In [13]:
NYCstarbucks = vis.mongodb_to_df("NYCstarbucks")
NYCbaskStadium = vis.mongodb_to_df("NYCbaskStadium")
NYCschools = vis.mongodb_to_df("NYCschools")
NYCairport = vis.mongodb_to_df("NYCairport")
NYCtrainStations = vis.mongodb_to_df("NYCtrain_stations")
NYCpet_grooming = vis.mongodb_to_df("NYCpet_grooming")
NYCdog_parks = vis.mongodb_to_df("NYCdog_parks")
NYCbars_clubs = vis.mongodb_to_df("NYCbars_clubs")
NYCvegan_vegetarian = vis.mongodb_to_df("NYCvegan_vegetarian")

    - SEATTLE

In [16]:
STstarbucks = vis.mongodb_to_df("STstarbucks")
STbaskStadium = vis.mongodb_to_df("STbaskStadium")
STschools = vis.mongodb_to_df("STschools")
STairport = vis.mongodb_to_df("STairport")
STtrainStations = vis.mongodb_to_df("STtrain_stations")
STpet_grooming = vis.mongodb_to_df("STpet_grooming")
STdog_parks = vis.mongodb_to_df("STdog_parks")
STbars_clubs = vis.mongodb_to_df("STbars_clubs")
STvegan_vegetarian = vis.mongodb_to_df("STvegan_vegetarian")

    IMPORT AND READ GEOJSONS OF THE CITIES  

In [18]:
SF_gj = vis.read_geojson("data/SanFrancisco.Neighborhoods.json")
NYC_gj = vis.read_geojson("data/d085e2f8d0b54d4590b1e7d1f35594c1pediacitiesnycneighborhoods.geojson")
ST_gj = vis.read_geojson("data/Neighborhood_Map_Atlas_Neighborhoods.geojson")

- HEAT MAPS TO COMPARE DENSITY OF REQUIREMENTS IN EACH CITY

    SAN FRANCISCO

    NEW YORK CITY

    SEATTLE